# In actual practical test itself, the questions will be in MCQ format with open-ended questions.

This is mock practical test section B is to let you practice your coding <br>
Download the hdb data set here https://data.gov.sg/datasets/d_8b84c4ee58e3cfc0ece0d773c8ca6abc/view<br>
Run the following code block before you begin answering the questions

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

In [2]:

def storey_checker(txt,storey_threshold):
    temp=[int(s) for s in txt.split() if s.isdigit()]
    return all([i<storey_threshold for i in temp])

In [3]:
df=pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

# The following pre-processing is to make the dataset more manageable for this mock only

#process blocks as integers, removing the last character if it's a letter
df['block']=df['block'].apply(lambda x: int(x[:-1]) if x[-1].isalpha() else int(x))

#keep rows with ['SENGKANG', 'BISHAN', 'WOODLANDS', 'TAMPINES', 'BEDOK', 'PUNGGOL']
df = df[df['town'].isin(['TAMPINES', 'BEDOK', 'PUNGGOL'])]

#keep rows with the word 'ROOM' in the flat_type column
df = df[df['flat_type'].str.contains('ROOM', case=False)]

#keep rows with integer values in storey_range less than 10
df = df[df['storey_range'].apply(lambda x: storey_checker(x, 10))]

df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
56,2017-01,BEDOK,2 ROOM,101,BEDOK NTH AVE 4,04 TO 06,45.0,Improved,1978,60 years 06 months,238000.0
57,2017-01,BEDOK,3 ROOM,548,BEDOK NTH AVE 1,04 TO 06,68.0,New Generation,1980,62 years 06 months,272000.0
58,2017-01,BEDOK,3 ROOM,75,BEDOK NTH RD,07 TO 09,59.0,Improved,1978,60 years,278000.0
59,2017-01,BEDOK,3 ROOM,550,BEDOK NTH AVE 1,01 TO 03,68.0,New Generation,1980,62 years 05 months,280000.0
60,2017-01,BEDOK,3 ROOM,1,BEDOK STH AVE 1,01 TO 03,59.0,Improved,1976,58 years,280000.0
...,...,...,...,...,...,...,...,...,...,...,...
205835,2025-05,TAMPINES,5 ROOM,876,TAMPINES ST 84,04 TO 06,122.0,Improved,1988,62 years 03 months,740000.0
205836,2025-01,TAMPINES,5 ROOM,871,TAMPINES ST 86,04 TO 06,114.0,Improved,2017,91 years 03 months,840000.0
205837,2025-01,TAMPINES,5 ROOM,871,TAMPINES ST 86,04 TO 06,114.0,Improved,2017,91 years 02 months,812000.0
205839,2025-04,TAMPINES,5 ROOM,932,TAMPINES ST 91,04 TO 06,138.0,Model A,1985,59 years 09 months,808000.0


What “Town” occured the most number of times?

In [4]:
df.value_counts("town")

# Tampines

town
TAMPINES    9413
PUNGGOL     7456
BEDOK       6878
Name: count, dtype: int64

Drop the following columns, 'month', 'street_name', 'flat_model', and 'remaining_lease'

In [5]:
df = df.drop(['month', 'street_name', 'flat_model', 'remaining_lease'], axis = 1 )

Perform one hot encoding on required remaining columns.

In [6]:
df

,town,flat_type,block,storey_range,floor_area_sqm,lease_commence_date,resale_price
56,BEDOK,2 ROOM,101,04 TO 06,45.0,1978,238000.0
57,BEDOK,3 ROOM,548,04 TO 06,68.0,1980,272000.0
58,BEDOK,3 ROOM,75,07 TO 09,59.0,1978,278000.0
59,BEDOK,3 ROOM,550,01 TO 03,68.0,1980,280000.0
60,BEDOK,3 ROOM,1,01 TO 03,59.0,1976,280000.0
...,...,...,...,...,...,...,...
205835,TAMPINES,5 ROOM,876,04 TO 06,122.0,1988,740000.0
205836,TAMPINES,5 ROOM,871,04 TO 06,114.0,2017,840000.0
205837,TAMPINES,5 ROOM,871,04 TO 06,114.0,2017,812000.0
205839,TAMPINES,5 ROOM,932,04 TO 06,138.0,1985,808000.0


In [7]:
df_ohn = df.copy()
df = pd.get_dummies(df,columns=["town","flat_type","storey_range"])

In [8]:
df

,block,floor_area_sqm,lease_commence_date,resale_price,town_BEDOK,town_PUNGGOL,town_TAMPINES,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,storey_range_01 TO 03,storey_range_04 TO 06,storey_range_07 TO 09
56,101,45.0,1978,238000.0,True,False,False,True,False,False,False,False,True,False
57,548,68.0,1980,272000.0,True,False,False,False,True,False,False,False,True,False
58,75,59.0,1978,278000.0,True,False,False,False,True,False,False,False,False,True
59,550,68.0,1980,280000.0,True,False,False,False,True,False,False,True,False,False
60,1,59.0,1976,280000.0,True,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205835,876,122.0,1988,740000.0,False,False,True,False,False,False,True,False,True,False
205836,871,114.0,2017,840000.0,False,False,True,False,False,False,True,False,True,False
205837,871,114.0,2017,812000.0,False,False,True,False,False,False,True,False,True,False
205839,932,138.0,1985,808000.0,False,False,True,False,False,False,True,False,True,False


Train test split the data using random_state=5, using 20% of the data as testing.

In [9]:
random_state = 5
x = df.drop('resale_price',axis = 1)
y = df['resale_price']

X_train, X_test, y_train,y_test = train_test_split(x,y,random_state=random_state,test_size=0.2)

Train a linear regression model

In [10]:
model = LinearRegression()

model.fit(X_train,y_train)

LinearRegression()

Calculate the mean absolute error of the test set.

In [11]:
y_pred = model.predict(X_test)

MAE = np.mean(np.abs(y_pred - y_test))
print(F"The MAE: {MAE}")

The MAE: 65501.35614220677


Using the trained linear regression model, predict the price of a property 
- In town Bedok
- 4 room flat
- Block 532
- 89 sqm floor area
- Bedok North Street 3, Singapore 460532
- On the 5th storey
- Lease commenced in 2011	
- Remaining Lease: 93 years 07 months


In [12]:
df.columns

Index(['block', 'floor_area_sqm', 'lease_commence_date', 'resale_price',
       'town_BEDOK', 'town_PUNGGOL', 'town_TAMPINES', 'flat_type_2 ROOM',
       'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM',
       'storey_range_01 TO 03', 'storey_range_04 TO 06',
       'storey_range_07 TO 09'],
      dtype='object')

In [ ]:
X_unseen =[
532,
89,
2011,
1,
0,
0,
0,
0,
1,
0,
0,
1,
0,

]

In [14]:
y_unseen_pred = model.predict([X_unseen])
y_unseen_pred

C:\Users\Ryan Ng\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([622400.54194728])

## Open ended
In HDB dataset, ‘Block’ was found to have low feature importance using the default RandomForestRegressor model. What can be done to the column and why?

In [15]:
drop the block columns since it has low feature importance  meaning that it is not important factor in predicting the house price 

SyntaxError: invalid syntax (581946745.py, line 1)

In [ ]:
model = joblib.dump(model, 'model.pkl')